Here are some definitions needed for the computations

In [2]:
r = 3
n = 3
x, z = var('x z')
indices = [(i,j) for i in range(n) for j in range(n)]

In [3]:
a_var = [var('a_{}_{}'.format(i,j), latex_name='a_{{{},{}}}'.format(i,j)) for (i,j) in indices]
a = dict(zip(indices, a_var))

In [4]:
b_var = [var('b_{}_{}'.format(i,j), latex_name='b_{{{},{}}}'.format(i,j)) for (i,j) in indices]
b = dict(zip(indices, b_var))

In [5]:
c_var = [var('c_{}_{}'.format(i,j), latex_name='c_{{{},{}}}'.format(i,j)) for (i,j) in indices]
c = dict(zip(indices, c_var))

In [6]:
def ai(soort, index):
    i0, j0 = index
    if soort == 1:
        i = i0
        j = j0
        return a[i, j] / c[i, j]
    elif soort == 2:
        i = 2 * n - j0 - 1
        j = i0 + i - n
        return b[i % r, j % r] / a[i % r, j % r]
    elif soort == 3:
        if j0 + 1 >= i0:
            i = n + j0 - i0
            j = 2 * n - j0 - 1 + i - n
        else:
            i = n + j0 - i0
            j = 2 * n - i0 - 1
        return c[i % r, j % r] / b[i % r, j % r]

In [7]:
def bi(soort, index):
    i0, j0 = index
    if soort == 1:
        i = i0
        j = j0
        return b[i, j] / c[i, j]
    elif soort == 2:
        i = 2 * n - j0 - 1
        j = i0 + i - n
        return c[i % r, j % r] / a[i % r, j % r]
    elif soort == 3:
        if j0 + 1 >= i0:
            i = n + j0 - i0
            j = 2 * n - j0 - 1 + i - n
        else:
            i = n + j0 - i0
            j = 2 * n - i0 - 1
        return a[i % r, j % r] / b[i % r, j % r]

In [8]:
def T(m, var, soort):
    matrix = zero_matrix(SR, n); matrix
    for k in range(0, n):
        if k != n - 1:
            matrix[k, k] = ai(soort, (m - 1, k))
            matrix[k, (k + 1) % n] = bi(soort, (m - 1, k))
        else:
            matrix[k, k] = ai(soort, (m - 1, k))
            matrix[k, (k + 1) % n] = bi(soort, (m - 1, k)) * var
    return matrix

In [9]:
def W(var, soort):
    W = 1
    for i in range(1, r+1):
        W = W * T(i, var, soort)
    return W

In [10]:
def writeMatrix(matrix):
    newstring = "\\begin{pmatrix}\n"
    n, m = matrix.dimensions()
    for i in range(n):
        newstring = newstring + "   "
        for j in range(m):
            newstring = newstring + latex(matrix[i, j]) + " "
            if j != m - 1:
                newstring = newstring + "& "
        if i != n-1:
            newstring = newstring + "\\\\" + "\n"
        else:
            newstring = newstring + "\n"
    newstring = newstring + "\\end{pmatrix}"
    return newstring

In [11]:
B_var = [var('B_{}_{}'.format(i,j), latex_name='B_{{{},{}}}'.format(i,j)) for (i,j) in indices]
B_vals = dict(zip(indices, B_var))
B = Matrix(SR, n, n, B_vals)

In [12]:
C_var = [var('C_{}_{}'.format(i,j), latex_name='C_{{{},{}}}'.format(i,j)) for (i,j) in indices]
C_vals = dict(zip(indices, C_var))
C = Matrix(SR, n, n, C_vals)

In [13]:
D_var = [var('D_{}_{}'.format(i,j), latex_name='D_{{{},{}}}'.format(i,j)) for (i,j) in indices]
D_vals = dict(zip(indices, D_var))
D = Matrix(SR, n, n, D_vals)

We want the following three matrix equations to hold

In [14]:
M1 = T(1, z, 1) - T(1, z, 2)*B
M2 = B*T(2, z, 1) - T(2, z, 2)*C
M3 = C*T(3, z, 1) - T(3, z, 2)*D

Let us split the equations inside the matrix to find B, C and D

In [15]:
eq1 = []
eq2 = []
eq3 = []
varB = []
varC = []
varD = []
for i in range(n):
    for j in range(n):
        eq1.append(M1[i, j])
        eq2.append(M2[i, j])
        eq3.append(M3[i, j])
        varB.append(B[i, j])
        varC.append(C[i, j])
        varD.append(D[i, j])

In [16]:
def writeEqSet(number):
    code = "\\begin{cases}\n"
    if number == 1:
        for eq in eq1:
            code = code + "   " + latex(eq) + " = 0" + '\\\\\n'
    elif number == 2:
        for eq in eq2:
            code = code + "   " + latex(eq) + " = 0" + '\\\\\n'
    else:
        for eq in eq3:
            code = code + "   " + latex(eq) + " = 0" + '\\\\\n'
    code = code + "\\end{cases}"
    return code

We solve for B

In [18]:
solB = solve(eq1, varB)
B = B.subs(solB)

We solve for C. Remark, C is dependent on B

In [19]:
solC = solve(eq2, varC)
C = C.subs(solC)

We solve for D. Remark, D is dependent on C and thus of B

In [20]:
solD = solve(eq3, varD)
D = D.subs(solD)

In [22]:
W1 = W(z, 1)
W2 = W(z, 2)
W3 = W(z, 3)

indexen = [(i,j) for i in range(1, n+1) for j in range(1, n+1)]
T_var = [var('Ts_{}_{}'.format(i,j)) for (i,j) in indices]
Ts = dict(zip(indices, a_var))
for i in range(1, n+1):
    for j in range(1, n+1):
        Ts[i,j] = T(i, z, j)

We check that the conditions indeed hold

In [23]:
C = C.subs(solB)
D = D.subs(solC)
D = D.subs(solB)

show(Ts[1, 1] == Ts[1, 2]*B)
show(B*Ts[2, 1] == Ts[2, 2]*C)
show(C*Ts[3, 1] == Ts[3, 2]*D)

True

True

True

Lastly, we were searching D such that the following holds

In [24]:
show(W1 == W2*D)

True

We cannot conjugate them, because their characteristic polynomials differ

In [26]:
def comparePoly(p, q):
    dp = p.degree()
    dq = q.degree()

    coeffp = p.coefficients(sparse=False)
    coeffq = q.coefficients(sparse=False)
    if dp != dq:
        return False
    else:
        for i in range(dp):
            if coeffp[i] != coeffq[i]:
                return False
    return True

In [28]:
p1 = W1.charpoly()
p2 = W2.charpoly()

show(comparePoly(p1, p2))

False